# SynthEval user guide
This notebook demonstrates how the included software is used for evaluating synthetic data.

The example is based on the Palmer Archipelago penguin data from kaggle (https://www.kaggle.com/datasets/parulpandey/palmer-archipelago-antarctica-penguin-data?select=penguins_size.csv), and the synthetic version is generated using DataSynthesizer with default parameters.

The current version of the code requires the following imports;

In [8]:
import pandas as pd

from src.syntheval import SynthEval

ImportError: attempted relative import with no known parent package

In [2]:
### First we load and prepare the data
load_dir = 'example/'
filename = 'penguins'

target_column = 'species'                           # column to use as target for classification metrics and coloration of PCA plot.
categorical_columns = ['species','island','sex']    # Categorical columns can be either supplied or automatically inferred using a number of unique values threshold.

df_real = pd.read_csv(load_dir + filename + '_train.csv')
df_fake = pd.read_csv(load_dir + filename + '_BN_syn.csv').round(1) # be sure to round to an appropriate number of decimals

### Testing data is not required, but the usability analysis will be more complete if it is included. 
df_test = pd.read_csv(load_dir + filename + '_test.csv')

In the first example, we show the minimalist code required to start evaluating synthetic data. 

In [4]:
### First SynthEval object is created then run with the "full_eval" presets file.
S = SynthEval(df_real)
S.evaluate(df_fake,target_column,presets_file="F:/GitHub repositories/syntheval/src/syntheval/presets/full_eval.json")

SynthEval: inferred categorical columns...
SynthEval: synthetic data read successfully
Unrecognised keyword: dwm
Unrecognised keyword: plot_pca
Unrecognised keyword: cio
Unrecognised keyword: corr_diff
Unrecognised keyword: mi_diff
Unrecognised keyword: ks_test
Unrecognised keyword: h_dist
Unrecognised keyword: p_mse
Unrecognised keyword: nndr
Unrecognised keyword: nnaa
Unrecognised keyword: cls_acc
Unrecognised keyword: dcr
Unrecognised keyword: hit_rate
Unrecognised keyword: eps_risk

SynthEval results

+---------------------------------------------------------------+
+---------------------------------------------------------------+


{}

In [ ]:
### Specify columns
class_cat_col = ['species','island','sex']
class_lab_col = 'species'

In [ ]:
### The full version of the benchmark is run using the full_eval() method, again with a set of fake samples
evaluator.full_eval(df_fake,class_lab_col)